# Purpose

Currently, I'm working on a side project using NYC COVID-19 data provided by the DOHMH. It's a time series spanning the last several years. The current structure is suitable for my needs but, as I was working on it, I wondered at restructuring. 

The data is wide and makes my analyses tedious. This doesn't need to be the case. Since the dataset is large, spreadsheets lag too much to be very useful. A Python script made the most sense to me. 

A longer dataset with less columns. That's the idea. It would certainly clean my SQL queries up. Maybe I'll generalize the process here in case I need it again. I don't anticipate needing more than a few loops. We'll see. 

**Ben Salis**

In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
cov_count = pd.read_csv("/kaggle/input/covid-19-daily-counts-nyc/COVID-19_Daily_Counts_of_Cases__Hospitalizations__and_Deaths.csv")

In [ ]:
cov_count.head()
cov_header = list(cov_count.columns)
print(len(cov_header))

In [ ]:
# Only want specific attributes for now. Culling this list should make my cases simpler.
for item in cov_header:
    if 'PROBABLE' in item: #I want more information on these columns. 
        cov_header.remove(item)
    elif 'AVG' in item:
        cov_header.remove(item)
        
print(cov_header)
print(len(cov_header))

In [ ]:
pre_df = [] # list that will be made into a dataframe by the end

In [ ]:
'''
Using this nested loop to restructure the raw data. 
Honestly, making a method might be worthwhile. This
isn't the first time I've needed to pivot a large dataset
in this way. 
'''
for elem in cov_header:
    for row in range(len(cov_count)):
        date = cov_count['date_of_interest'][row]
        val = cov_count[elem][row]
        if elem.startswith('SI') == True: #could use "in" operator. 
            borough = 'Staten Island'
            if 'CASE' in elem:
                category = 'Cases'
                pre_df += [(date,borough,category,val)]
            elif 'DEATH' in elem:
                category = 'Deaths'
                pre_df += [(date,borough,category,val)]
            elif 'HOSPITALIZED' in elem:
                category = 'Hospitalized'
                pre_df += [(date,borough,category,val)]

In [ ]:
print(len(pre_df))

In [ ]:
df = pd.DataFrame(pre_df, columns= ["Date","Location", "Classification", "Count"])
df #this long data format is better for SQL and Tableau. The wide version wasn't great

# Note: Could keep analysis in Python, but this script is primarily a proof of concept

In [ ]:
df.to_csv('COVID19_count_long.csv',index = False)